<a href="https://colab.research.google.com/github/ciepielajan/SDA_SMOG_Wawelski/blob/main/Jan/Krakow_siatka_punktow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install folium

In [2]:
import json

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import folium
from folium.plugins import MarkerCluster

In [118]:

location = [50.0614 ,19.9366]
left_down_corner =  [49.99 ,19.82]

m = folium.Map(location=location, zoom_start=11)

for i in range(25):
  i = i * 0.01
  for x in range(20):
    x = x * 0.006
    folium.Marker([left_down_corner[0] + x, left_down_corner[1]+i]).add_to(m)

m